In [1]:
# https://docs.microsoft.com/en-us/azure/azure-databricks/databricks-stream-from-eventhubs
# https://github.com/Azure/azure-event-hubs-spark/blob/master/docs/PySpark/structured-streaming-pyspark.md

connectionString = "Endpoint=sb://ihsuprodblres058dednamespace.servicebus.windows.net/;SharedAccessKeyName=iothubowner;SharedAccessKey=1hiERMO+U01xjClcFab/Xk2nRGWYTCP6X5a2QnMEj/Y=;EntityPath=iothub-ehub-contosohub-448302-7369b6590a"

ehConf = {
  'eventhubs.connectionString' : connectionString
}

df = spark \
  .readStream \
  .format("eventhubs") \
  .options(**ehConf) \
  .load()
  
data = df.selectExpr("CAST(body as STRING)")

from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType
jsDf = data.select(get_json_object(data['body'],"$.Id").alias('DeviceId'),
                               get_json_object(data['body'],"$.X").alias('X').cast(DoubleType()),
                               get_json_object(data['body'],"$.Y").alias('Y').cast(DoubleType()),
                               get_json_object(data['body'],"$.Z").alias('Z').cast(DoubleType()))

fell = jsDf.filter(jsDf['Y'] == 0.5)

writeConnectionString = "Endpoint=sb://ndccontosons.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=cbBE7Zx8/nCxcStHlF+qteYNV6DM5ohZkCBr5wYkCJY=;EntityPath=ndccontosoeh"
ehWriteConf = {
  'eventhubs.connectionString' : writeConnectionString
}

ds = df \
  .select("body") \
  .writeStream \
  .format("eventhubs") \
  .options(**ehWriteConf) \
  .option("checkpointLocation", "///output.txt") \
  .start()